# 使用backtrader尝试模拟交易

In [1]:
import pandas as pd  
from datetime import datetime
import backtrader as bt
import matplotlib.pyplot as plt
%matplotlib inline

## 指定策略

In [5]:
class MyStrategy(bt.Strategy):
    
    #全局设定交易策略的参数
    params=(
        ('maperiod',20),
           )

    def __init__(self):
        #指定价格序列
        self.dataclose=self.datas[0].close
        # 初始化交易指令、买卖价格和手续费
        self.order = None
        self.buyprice = None
        self.buycomm = None

        #添加移动均线指标，内置了talib模块
        self.sma = bt.indicators.SimpleMovingAverage(
                      self.datas[0], period=self.params.maperiod)
    def next(self):
        print(self.dataclose[0], self.sma[0])
        if self.order: # 检查是否有指令等待执行, 
            return
        # 检查是否持仓   
        if not self.position: # 没有持仓
            #执行买入条件判断：收盘价格上涨突破20日均线
            if self.dataclose[0] > self.sma[0]:
                #执行买入
                self.order = self.buy(size=500)         
        else:
            #执行卖出条件判断：收盘价格跌破20日均线
            if self.dataclose[0] < self.sma[0]:
                #执行卖出
                self.order = self.sell(size=500)

In [3]:
data = bt.feeds.GenericCSVData(
    dataname='./predictions/all_trading_check_point_all.csv',

    fromdate=datetime(2016,9,12),
    todate=datetime(2017,9,12),

    dtformat=('%Y-%m-%d'),


    datetime=1,
    time=-1,
    high=-1,
    low=-1,
    open=-1,
    close=2,
    volume=-1,
    openintreset=-1
)
data

In [6]:
cerebro = bt.Cerebro()
cerebro.adddata(data)
cerebro.resampledata(data,timeframe=bt.TimeFrame.Days)
cerebro.broker.setcash(1000.0)
cerebro.broker.setcommission(commission=0.001)
cerebro.addstrategy(MyStrategy)

assets_before = cerebro.broker.getvalue()
cerebro.run()
assets_after = cerebro.broker.getvalue()

assets_before, assets_after

hello


(1000.0, 1000.0)